In [20]:
from nltk.parse.corenlp import CoreNLPDependencyParser
import nltk

parser = CoreNLPDependencyParser()

def calculate_frequency(aspects):
    aspect_frequency = {}
    
    for aspect in aspects:
        if(aspect in aspect_frequency):
            aspect_frequency[aspect] += 1
        else:
            aspect_frequency[aspect] = 1
            
    return aspect_frequency

k_DP = ['conj']
def pruning_based_on_clause(aspect_frequency, reviews, predictions):
    pruning = []
    index = 0
    for review in reviews:
        temp = predictions[index]
        index += 1
        if isinstance(temp, str):
            prediction = temp.split('|')
        else:
            continue
        parse = next(parser.raw_parse(review))
        for (w1, dep, w2) in list(parse.triples()):
            if(w1[0] in aspect_frequency and w2[0] in aspect_frequency):
                if(dep not in k_DP):
                    if(aspect_frequency[w1[0]] > aspect_frequency[w2[0]]):
                        if w2[0] in prediction:
                            pruning.append(w2[0])
                    elif(aspect_frequency[w1[0]] < aspect_frequency[w2[0]]):
                        if w1[0] in prediction:
                            pruning.append(w1[0])
                
    return pruning
def pruning_based_other_products_and_dealers(aspect_frequency, reviews,predictions, window=3):
    pruning = []
    ProductINDI = ["compare to", "compare with", "better than", "worse than"]
    DealerINDI  = ["shop with", "buy from"]
    count = 0
    for review in reviews:
        temp = predictions[count]
        count += 1
        if isinstance(temp, str):
            prediction = temp.split('|')
        else:
            continue
        if any(indication in review for indication in ProductINDI):
            tokens = nltk.word_tokenize(review)
            index = 0
            while index < len(tokens) - 1:
                if tokens[index] + " " + tokens[index + 1] in ProductINDI:
                    index += 2
                    for x in range(window):
                        next_window = index + x + 1;
                        if next_window < len(tokens) and tokens[next_window] in aspect_frequency:
                            if tokens[next_window] in prediction:
                                pruning.append(tokens[next_window])
                else :
                    index += 1
                    
        if any(indication in review for indication in DealerINDI):
            tokens = nltk.word_tokenize(review)
            index = 0
            while index < len(tokens) - 1:
                if tokens[index] + " " + tokens[index + 1] in DealerINDI:
                    index += 2
                    for x in range(window):
                        next_window = index + x + 1;
                        if next_window < len(tokens) and tokens[next_window] in aspect_frequency:
                            if tokens[next_window] in prediction:
                                pruning.append(tokens[next_window])
                else :
                    index += 1 
                        
    return pruning

In [106]:
import pandas as pd
import numpy as np

def calculate_result(save_file):
    hs = pd.read_csv(save_file)
    dp_aspect = []

    for t in hs['prediction']:
        if t is not np.nan:
            for s in t.split('|'):
                dp_aspect.append(s.split('!')[0])

    target = []
    for t in hs['aspect']:
        if t is not np.nan:
            for s in t.split('|'):
                target.append(s.split('!')[0])

    tp = calculate_frequency(target)
    ap = calculate_frequency(dp_aspect)
    
    pruning_clause = pruning_based_on_clause(ap,hs['0'], hs['prediction'])
    prun_dealer_product = pruning_based_other_products_and_dealers(ap, hs['0'], hs['prediction'])

    a = pruning_clause + prun_dealer_product
    
    for x in list(dict.fromkeys(dp_aspect)):
        flag = True
#         splitted = x.split('-')
#         if len(splitted) > 1:
#             for word in splitted:
#                 for match in list(dict.fromkeys(dp_aspect)):
#                     if word in match and word != match:
#                         if ap[match] > 1:
#             flag = False

        if ap[x] < 3 and flag:
            ap[x] = 0

    for freq in list(dict.fromkeys(a)):
        try:
            if ap[freq] > 0:
                ap[freq] -= 1
        except: continue
            
    new_dp_aspect = []
    for a in ap:
        if ap[a] > 0:
            new_dp_aspect.append(a)
#     print(new_dp_aspect)


    atas_recall = 0
    atas_precision = 0
    bawah_precision = 0
    bawah_recall = 0

    new_target = []
    for x in list(dict.fromkeys(target)):
#         print(x)
        for y in x.split(' '):
            new_target.append(y.split('!')[0])

    new_aspect = []
    for x in list(dict.fromkeys(new_dp_aspect)):
        for y in x.split('-'):
            new_aspect.append(y.split('!')[0])

    for x in tp:
        bawah_precision += tp[x]
        flag = False            
        for y in x.split(' '):            
            for apt in new_aspect:
                if y in apt:
                    atas_precision += tp[x]
                    flag = True
                    break
            if flag:
                break
        if not flag:
            print('t', x, tp[x])

    for x in ap:
        if ap[x] != 0:
            bawah_recall += ap[x]
            flag = False
            for y in x.split('-'):            
                for tgt in new_target:
                    if y in tgt:
                        atas_recall += ap[x]
                        flag = True
                        break
                if flag:
                    break
            if not flag:
                print(x, ap[x])

    precision = atas_precision/bawah_precision
    recall = atas_recall/bawah_recall
#     print(precision, recall)
    f1 = (2 * precision * recall) / (precision + recall)

    return precision, recall, f1

In [107]:
filename = [
#             'Apex AD2600 Progressive-scan DVD player',
            'Canon G3',
#             'Creative Labs Nomad Jukebox Zen Xtra 40GB',
#             'Nikon coolpix 4300',
#             'Nokia 6610'
]

path = [
    'hasil',
#     'hasil_plus',
    'hasil_chunk',
#     'hasil_chunk_plus',
    'hasil_clause'
]

df = pd.DataFrame(columns=['name','type', 'P', 'R', 'F'])

for directory in path:
    for file in filename:
        P, R, F = calculate_result(directory + '/' + file + '.csv')
        print(P, R, F)
        df = df.append({'name': file, 'type': directory, 'P': P, 'R': R, 'F': F}, ignore_index=True)
        df.to_csv("result-all.csv")
        df.to_excel("result-all.xlsx")

t use 9
t feature 7
t option 2
t speed 1
t metering option 1
t spot metering 1
t 4mp 1
t size 2
t design 3
t menu 2
t canera 1
t print 1
t feel 3
t product 1
t zooming lever 1
t color 2
t grain 1
t depth 1
t service 1
t automode 1
t raw format 1
t shape 1
t white offset 1
t unresponsiveness 1
t delay 2
t shoot 1
t body 2
t casing 1
t performance 1
t look 2
t finish 1
t tiff format 1
t lag 1
t import 1
t display 1
t compactflash 1
t noise 1
t made 1
t lever 1
t learning 1
t function 1
t macro 1
t distortion 1
t remote 1
t highlight 1
t no-off button 1
t download 1
t optic 1
t weight 1
fact 3
box 3
g2 7
line 3
ability 3
anyone 3
flexibility 3
choice 4
type 4
shutter 3
photography 6
review 4
range 7
market 4
thing 9
difference 3
olympus 3
bit 3
priority 3
lot 4
point 9
way 3
film 4
slr 5
digicam 3
technology 4
anything 3
screen 4
coolpix 3
unit 4
0.7098039215686275 0.7980456026058632 0.7513427424137172
t use 9
t feature 7
t option 2
t dial 2
t speed 1
t auto setting 2
t darn diopter adjus

In [108]:
# 0.7098039215686275 0.7980456026058632 0.7513427424137172
# 0.6588235294117647 0.8285714285714286 0.7340112994350283
# 0.7098039215686275 0.7887640449438202 0.7472037635989415

In [ ]:
0.8156862745098039 0.7162162162162162 0.7627218320821946
0.7764705882352941 0.7451327433628319 0.7604789599726308
0.8235294117647058 0.696763202725724 0.7548612484345132